In [0]:
import spacy_udpipe
from google.colab import files
import string
import re
from wiki_ru_wordnet import WikiWordnet
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
wikiwordnet = WikiWordnet()

In [0]:
from google.colab import files


### Загрузка текстов

In [0]:
uploaded = files.upload()

Saving decript_clean.txt to decript_clean.txt


In [0]:
text = uploaded['decript_clean.txt'].decode('UTF-8')

In [0]:
text[:100]

'Библиотека основана в 1973 году. Расположена в деревне Аксёново Дмитриевского сельского поселения Га'

### Загрузка русского корпуса

In [0]:
spacy_udpipe.download("ru")
nlp = spacy_udpipe.load("ru")

Already downloaded a model for the 'ru' language


### Тестирование

In [0]:
# is_digit

In [0]:
examples = ['Библиотека основана в 1973 году.', 'Малышевская сельская библиотека была основана в 1936 году', 'Библиотека № 1 была открыта в феврале 1968 года', 'Дата основания районной библиотеки – 1983 год.']

In [0]:
# is_digit

def view_structure(text):
  
  doc = nlp(text)

  for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.head)

In [0]:
for ex in examples:
  view_structure(ex)
  print()

Библиотека библиотека NOUN nsubj:pass основана
основана основать VERB ROOT основана
в в ADP case году
1973 1973 ADJ amod году
году год NOUN obl основана
. . PUNCT punct основана

Малышевская Малышевский ADJ amod библиотека
сельская сельский ADJ amod библиотека
библиотека библиотека NOUN nsubj:pass основана
была быть AUX aux:pass основана
основана основать VERB ROOT основана
в в ADP case году
1936 1936 ADJ amod году
году год NOUN obl основана

Библиотека библиотека NOUN nsubj:pass открыта
№ № SYM nmod Библиотека
1 1 NUM nummod №
была быть AUX aux:pass открыта
открыта открыть VERB ROOT открыта
в в ADP case феврале
феврале февраль NOUN obl открыта
1968 1968 ADJ amod года
года год NOUN nmod феврале

Дата дата NOUN ROOT Дата
основания основание NOUN nmod Дата
районной районный ADJ amod библиотеки
библиотеки библиотека NOUN nmod основания
– – PUNCT punct год
1983 1983 ADJ amod год
год год NOUN nmod Дата
. . PUNCT punct Дата



***Схема:*** основана -> году -> 1936

In [0]:
view_structure('Дата основания районной библиотеки – 1983 год')

Дата дата NOUN ROOT Дата
основания основание NOUN nmod Дата
районной районный ADJ amod библиотеки
библиотеки библиотека NOUN nmod основания
– – PUNCT punct Дата
1983 1983 ADJ amod год
год год NOUN nmod Дата


### Тестирование Wordnet для русского

In [0]:
def get_syn(word):
  
  syns = []
  
  synsets = wikiwordnet.get_synsets(word)
  for index, item in enumerate(synsets):
    for w in item.get_words():
      syns.append(w.lemma())
  
  return syns

In [0]:
wordnet_ex = ['основывать', 'основать', 'открыть', 'находиться']


for word in wordnet_ex:
  syns = get_syn(word)
  print(syns)

['основывать', 'основывать']
['основать', 'основать', 'учредить']
['открыть', 'открыть', 'открыть']
['находиться', 'находиться', 'располагаться', 'находиться', 'находиться', 'обретаться']


In [0]:
get_syn('основать')

0 основать
1 основать
1 учредить


### Дата основания

* ('была', 'открыта', 'в')
* ('библиотека', 'была', 'открыта')
* ('библиотека', 'основана', 'в')
* ('была', 'основана', 'в')
* ('библиотека', 'была', 'основана')
* Дата рождения Всеволожской городской библиотеки – 19 мая 1908 года. 
* Свою историю библиотека ведет с 1932 года
* В 1945 году в здании клуба была организована сельская библиотека
* Официальная дата рождения библиотеки 18 июля 1954 года
* Дата основания районной библиотеки – 1983 год.
* Пищалкинская сельская библиотека работает в поселке Пищалкино с 1987 года. 

---> открыта, основана

Прогоним получившиеся слова для темы через ворднет, чтобы достать синонимы.

In [0]:
words = {'основана': ['библиотека', 'году', 'в'],
 'открыта': ['библиотека', 'году', 'в'],
 'организована': ['библиотека', 'году', 'в'],
 'учреждена': ['библиотека', 'году', 'в'],
 'работает': ['библиотека', 'года', 'с'],
 'открылась': ['библиотека', 'года'],
 'началась': ['библиотеки', 'году', 'в'],
 'возобновилась': ['библиотеки', 'году', 'в']}

In [0]:
def wordnet_syn_search(words):

    new_words = {}

    for word in words:
    
        morp = morph.parse(word)[0] 
        syns = get_syn(morp.normal_form)
    
        morp = str(morp.tag)
        morp = re.sub(' ', ',', morp)
        morp = set(morp.split(','))
    
        for syn in syns:
            w = morph.parse(syn)[0]
            w = w.inflect(morp)
                
            if w and w.word != word:
                new_words[w.word] = words[word]

    return new_words

In [0]:
new_words = wordnet_syn_search(words)
new_words

{'возникла': ['библиотеки', 'году', 'в'],
 'доступна': ['библиотека', 'году', 'в'],
 'настала': ['библиотеки', 'году', 'в'],
 'наступила': ['библиотеки', 'году', 'в'],
 'образована': ['библиотека', 'году', 'в'],
 'основана': ['библиотека', 'году', 'в'],
 'появилась': ['библиотеки', 'году', 'в'],
 'пришла': ['библиотеки', 'году', 'в'],
 'разверста': ['библиотека', 'году', 'в'],
 'учреждена': ['библиотека', 'году', 'в']}

Уже лучше, можем взять 'возникла', 'доступна', 'образована', 'появилась'.

In [0]:
new = ['возникла', 'доступна', 'образована', 'появилась']
for i in new:
  words[i] = new_words[i]

In [0]:
words['возникла'] = ['библиотека', 'году', 'в']
words['появилась'] = ['библиотека', 'году', 'в']

In [0]:
words

{'возникла': ['библиотека', 'году', 'в'],
 'возобновилась': ['библиотеки', 'году', 'в'],
 'доступна': ['библиотека', 'году', 'в'],
 'началась': ['библиотеки', 'году', 'в'],
 'образована': ['библиотека', 'году', 'в'],
 'организована': ['библиотека', 'году', 'в'],
 'основана': ['библиотека', 'году', 'в'],
 'открылась': ['библиотека', 'года'],
 'открыта': ['библиотека', 'году', 'в'],
 'появилась': ['библиотека', 'году', 'в'],
 'работает': ['библиотека', 'года', 'с'],
 'учреждена': ['библиотека', 'году', 'в']}

Теперь переходим к синтаксическому разбору.

and token.text in words[token.head]

In [0]:
text = 'В 1945 году в здании клуба была организована сельская библиотека'

doc = nlp(text.lower())

for token in doc:
  print(token.text)
  head = str(token.head)
  txt = str(token.text)
  print()
  
  if head in words and txt in words[head]:
    
    print(token.text, token.lemma_, token.pos_, token.dep_, token.head)

в
1945
году
году год NOUN obl организована
в
здании
клуба
была
организована
сельская
библиотека
библиотека библиотека NOUN nsubj:pass организована


In [0]:
'организована' in words

True